In [32]:
# 검색데이터가 딕셔너리에 있다면, 해당 value 값을 돌려주고, 없다면 구글검색을 통해서 데이터를 수집하고, 저장하고, 돌려준다.
# Chrome: 버전 90.0.4430.85(공식 빌드) (x86_64)

In [33]:
import re
import pandas as pd
import openpyxl
from urllib.parse import ParseResultBytes, quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver
import pickle

from tqdm import tqdm

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words = set(stopwords.words('english')) 

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

# UserAgent값을 바꿔줍시다! 서버가 인식하지 못하도록 가상으로 headless 값 추가함ㅠ
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")


In [34]:
def cleaning_data(input_data):
    remove = input_data.replace("  "," ") # 변환
    remove_ = re.sub(r"\t", " ", remove) # 제거
    remove__ = re.sub(r"\n", " ", remove_) # 제거
    remove__ = remove__.replace("   ", " ")
    remove__ = remove__.replace("  ", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace("…/", " ")
    remove__ = remove__.replace("…", " ")
    remove__ = remove__.replace("/", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace(")", ",")
    remove__ = remove__.replace("(", ",")
    preprossed = remove__.split(",") # 단어를 리스트로 변환
    #print(preprossed)
    
    # 표제어 추출, 동사는 현재형으로 변환
    lemma_list =[]
    for i in preprossed:
        lema_re = lemmatizer.lemmatize(i, pos='v') #표제어 추출, 동사는 현재형으로 변환
        lemma_list.append(lema_re)
    
    # 표제어 추출
    ext_lema = [lemmatizer.lemmatize(w) for w in preprossed]
    # 중복값을 제거하고
    rm_dupli = set(ext_lema)
    # 다시 리스트로 만들고
    re_li = list(rm_dupli)
    # 빈 값은 제거하고
    get_wd =list(filter(None, re_li))
    # 소문자로 모두 변환
    lower_wd = [i.lower() for i in get_wd]
    
    result = []
    for w in lower_wd: 
        if w not in stop_words: 
            result.append(w)
    return result

In [35]:
# 구글엔진을 이용해 키워드를 검색하고 결과 링크를 모두 조사하여 text 파일의 리스트로 결과 도출되는 코드
def check_searched_keywords(input_word):
    baseUrl = 'https://www.google.com/search?q='

    plusUrl = input_word


    # url = baseUrl + quote_plus(plusUrl)
    url = baseUrl + plusUrl
    # 한글을 사용할 경우 :  quote_plus 적용 - URL에 막 %CE%GD%EC 이런 거 생성해줌

    driver = webdriver.Chrome(executable_path= r'./data/chromedriver_office', chrome_options=options)
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, features="html.parser")

    v = soup.select('.yuRUbf')

    search_title_result = []
    search_linked_contents_result = []
    for i in v:
        #print(i.select_one('.LC20lb.DKV0Md').text)
        search_title_result.append(i.select_one('.LC20lb.DKV0Md').text)
        print(i.a.attrs['href'])
        search_linked_contents_result.append(i.a.attrs['href'])
        #print()


    # search_linked_contents_result 의 각 링크로 접속하여 해당 내용을 모두 text로 크롤링한 후, body내의 단어들만 추출한다.
    get_all_linked_web_data = []
    for linked_page in tqdm(search_linked_contents_result):
        driver.get(linked_page)
        html = driver.page_source
        get_all_data = BeautifulSoup(html, features="html.parser")
        get_all_linked_web_data.append(get_all_data)
        
    body = re.search('<body.*/body>', html, re.I|re.S)
    if (body is None):
        print ("No <body> in html")
        exit()
            
    body = body.group()
    #print(body)
    
    # 추출된 정보 클린징
    korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')#한글제거

    item_extract = str(body).replace('\n', ' ')
    item_extract = re.sub('<span.*?>.*?</span>', ' ', item_extract)
    item_extract = re.sub('<b>.*?</b>', ' ', item_extract)    
    item_extract = re.sub('<.*?>', ' ', item_extract)        
    item_extract = item_extract.replace('\t', ' ')
    item_extract = re.sub(korean, '', item_extract)
    item_extract = re.sub('[-=.#/?:$}]', ' ', item_extract)
    item_extract = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』;{}()'\\‘|\(\)\[\]\<\>`\'…》]", ' ', item_extract)
    #print (item_extract)
    
    driver.close()

    get_result_str = str(item_extract)
    result_cln = cleaning_data(get_result_str) # 결과값 청소
    result = list(set(result_cln)) # 중복제거

    dic_data = {'search keyword' : 'data_words'}
    dic_data[input_word] = result
    
    # 저장파일 이름 적용하기
    path = './data/college/' + input_word + '.pickle'
    
    # 저장하기
    with open(path, 'wb') as fw:
        pickle.dump(dic_data, fw)

    return dic_data

In [36]:
### 이것이 실행함수 ### 검색데이터가 딕셔너리에 있다면, 해당 value 값을 돌려주고, 없다면 구글검색을 통해서 데이터를 수집하고, 저장하고, 돌려준다.
def google_search_result_tp_knowledge():
    #load data
    
    input_word = input('무엇을 검색할까요? :')
    
    path = './data/college/' + input_word + '.pickle'
    
    dic_data = {'search keyword' : 'data_words'}
    
    # 저장할 파일 생성하기
    with open(path, 'wb') as fw:
        pickle.dump(dic_data, fw)
    
    # 저장파일 불러왜기(최초로 생성하였다면, 파일 안에는 아무것도 없기 때문에 google search 기능이 실행될거임)
    with open(path, 'rb') as fr:
        data_loaded = pickle.load(fr)

    #print('data_loaded :', data_loaded)
    #키가 있는지 검색, 즉 검색 단어가 있는지 저장된 딕셔너리에거 검색하기
    if input_word in data_loaded:
        print("key exist! value is : ", data_loaded[input_word]) # 키가 있다면 결과값을 추출(리턴) 해준다.
        result_fin = data_loaded[input_word]

    else:
        print("key not exist! so i am going to search data from google!") # 데이터에 검색어 키가 없기 때문에 구글검색 실행하여 데어터 수집하고 처리
        result_fin= check_searched_keywords(input_word)

    
    return result_fin

In [39]:
## run ##

result = google_search_result_tp_knowledge()

print(result)

무엇을 검색할까요? :UPENN Biology
key not exist! so i am going to search data from google!


/Users/kimkwangil/opt/anaconda3/envs/py37TF2/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':
  0%|          | 0/11 [00:00<?, ?it/s]

https://www.bio.upenn.edu/
https://www.collegefactual.com/colleges/university-of-pennsylvania/academic-life/academic-majors/biological-biomedical-sciences/
http://www.thedp.com/article/2017/08/out-of-89-majors-economics-and-biology-are-two-of-the-most-popular
https://en.wikipedia.org/wiki/University_of_Pennsylvania
https://www.ucas.com/explore/subjects/biological-sciences
https://catalog.upenn.edu/undergraduate/programs/biology-general-biology-ba/
https://catalog.upenn.edu/courses/biol/
https://www.med.upenn.edu/camb/
https://catalog.upenn.edu/undergraduate/programs/biology-molecular-cell-biology-ba/
https://catalog.upenn.edu/graduate/programs/biology-phd/
https://twitter.com/pennbiology


100%|██████████| 11/11 [00:31<00:00,  2.87s/it]

{'search keyword': 'data_words', 'UPENN Biology': ['xk', 'graphql_mutation_favorite_mode', 'responsive_web_search_reorder_typeahead_results_enabled', 'null', 'network_layer_503_backoff_mode', 'responsive_web_scroller_autopoll_enabled', 'ie', 'var', 'na', 'hashflags_animation_like_button_enabled', '2', 'gm', 'mk', 'responsive_web_use_app_prompt_enabled', 'vc', 'line', 'vn', 'responsive_web_graphql_likes_timeline', 'sg', 'detected', 'w', 'white', 'ee', 'entity', 'lu', 'responsive_web_instream_video_redesign_enabled', 'dm_conversations_muted_list_enabled', 'dm_reactions_config_active_reactions', 'vncdn', 'responsive_web_linger_refactor_enabled', 'mw', 'privacy', 'nz', 'a3', 'sha', 'hr', 'responsive_web_auto_logout_restrictedauthtoken_enabled', '😲', 'uaparsertags', 'livepipeline_client_enabled', 'responsive_web_dockable_autoplay_policy_enabled', 'identity_verification_intake_enabled', 'responsive_web_multiple_account_limit', 'dm_conversations_requests_inbox_actions_button_enabled', 'respon